In [40]:
!pip install -q bokeh

In [41]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [42]:
# rootAddr = '/content/drive/MyDrive/pytorch_Mask/data'

In [43]:
from torch.utils.data import DataLoader
from torch.utils.data import random_split
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets
from torchvision.datasets import ImageFolder


In [44]:
from bokeh.plotting import figure, show
from bokeh.io import show
from bokeh.models import LinearAxis, Range1d
import numpy as np
# import matplotlib.pyplot as plt
import bokeh.io

bokeh.io.output_notebook()
import bokeh.plotting


In [45]:
def data_loader():
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        # transforms.CenterCrop(256)   #中心剪裁后四周padding补充 (后续可以padding)
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.4990, 0.4567, 0.4188], std=[0.2913, 0.2778, 0.2836])
    ])
    trainset = ImageFolder('./data/train', transform=transform)
    #split train data
    # m=len(trainset)
    # train_data, val_data = random_split(trainset, [int(m-m*0.2), int(m*0.2)])

    train_loader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                               shuffle=True, num_workers=2)
    
    # valid_loader = torch.utils.data.DataLoader(val_data, batch_size=64)

    testset = ImageFolder('./data/test', transform=transform)
    test_loader = torch.utils.data.DataLoader(testset, batch_size=1000,
                                              shuffle=False, num_workers=2)

    return trainset, train_loader, trainset, test_loader



In [47]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_layer = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),  # in=3x256x256; out=32x256x256
            nn.ReLU(),
            nn.MaxPool2d(2),  # out=64x128x128
            nn.Conv2d(32, 16, kernel_size=3, padding=1),  # in=32x128x128, out=16x128x128
            nn.ReLU(),
            nn.MaxPool2d(2),  # out=16x64x64
            nn.Conv2d(16, 8, kernel_size=3, padding=1),  # in=16x64x64, out=8x64x64
            nn.ReLU(),
            nn.MaxPool2d(2)  # out=8x32x32
        )

        self.fc_layer = nn.Sequential(
            nn.Dropout(p=0.1),
            nn.Linear(8 * 32 * 32, 32 * 32),
            nn.ReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(32 * 32, 1024),
            nn.ReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(1024, 128),
            nn.ReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(128, 3)
        )

    def forward(self, x):
        # conv layer
        x = self.conv_layer(x)

        # flatten
        x = x.view(x.size(0), -1)

        # fc layer
        x = self.fc_layer(x)

        return x

In [48]:
def train_loop(num_epochs):
    total_step = len(train_loader)
    loss_list = []
    acc_list = []
    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(train_loader):
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss_list.append(loss.item())
            print(loss_list)
            # Backprop and optimisation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # Train accuracy
            total = labels.size(0)
            _, predicted = torch.max(outputs.data, 1)
            correct = (predicted == labels).sum().item()
            print(correct / total)
            acc_list.append(correct / total)
            print(i)
            if (i + 1) % 30 == 0:
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                    .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                        (correct / total) * 100))


In [49]:
if __name__ == '__main__':
    
    train_set,_, _,_ = data_loader()
    _,train_loader,_ , _ = data_loader()
    _,_,test_set,_ = data_loader()
    _, _, _, test_loader = data_loader()
    model = CNN()
    criterion = nn.CrossEntropyLoss()
    learning_rate = 0.001
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    num_epochs = 4
    trainLog = train_loop(num_epochs)

    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        print('Test Accuracy of the model on the test images: {} %'
              .format((correct / total) * 100))

    # tensorTrainLog = {'lossList': loss_list, 'accList': acc_list}    
    # return tensorTrainLog

      

[1.1195935010910034]
0.125
0
[1.1195935010910034, 1.0723567008972168]
0.4375
1
[1.1195935010910034, 1.0723567008972168, 1.0953210592269897]
0.46875
2
[1.1195935010910034, 1.0723567008972168, 1.0953210592269897, 1.0661133527755737]
0.53125
3
[1.1195935010910034, 1.0723567008972168, 1.0953210592269897, 1.0661133527755737, 1.0477356910705566]
0.5625
4
[1.1195935010910034, 1.0723567008972168, 1.0953210592269897, 1.0661133527755737, 1.0477356910705566, 1.0348013639450073]
0.5
5
[1.1195935010910034, 1.0723567008972168, 1.0953210592269897, 1.0661133527755737, 1.0477356910705566, 1.0348013639450073, 1.0154573917388916]
0.5
6
[1.1195935010910034, 1.0723567008972168, 1.0953210592269897, 1.0661133527755737, 1.0477356910705566, 1.0348013639450073, 1.0154573917388916, 0.937458872795105]
0.5625
7
[1.1195935010910034, 1.0723567008972168, 1.0953210592269897, 1.0661133527755737, 1.0477356910705566, 1.0348013639450073, 1.0154573917388916, 0.937458872795105, 0.9632304310798645]
0.5
8
[1.1195935010910034,

In [50]:
    
    # p = figure(y_axis_label='Loss', width=850, y_range=(0, 1), title='PyTorch ConvNet results')
    # p.extra_y_ranges = {'Accuracy': Range1d(start=0, end=100)}
    # p.add_layout(LinearAxis(y_range_name='Accuracy', axis_label='Accuracy (%)'), 'right')
    # p.line(np.arange(len(trainLog['lossList'])), trainLog['lossList'])
    # p.line(np.arange(len(trainLog['lossList'])), np.array(trainLog['accList']) * 100, y_range_name='Accuracy', color='red')

    # bokeh.plotting.show(p)

In [51]:
#Evaluate the model using skorch
!pip install skorch
from torch import nn
from skorch import NeuralNetClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from skorch import callbacks
from sklearn.metrics import confusion_matrix

In [53]:
class MaskConvNet():
  # model code  

  y_train = np.array([y for x, y in iter(train_set)])

  torch.manual_seed(0)

  net = NeuralNetClassifier(
      CNN,
      max_epochs=4,
      iterator_train__num_workers=4,
      iterator_valid__num_workers=4,
      lr=0.001,
      batch_size=32,
      optimizer = torch.optim.Adam,
      criterion=nn.CrossEntropyLoss,
      device=torch.device('cuda')
  )

  net.fit(train_set, y=y_train)

  y_pred = net.predict(test_set)
  y_test = np.array([y for x, y in iter(test_set)])
  print(accuracy_score(y_test, y_pred))
  # print(confusion_matrix(y_test, y_pred))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


  epoch    train_loss    valid_acc    valid_loss      dur
-------  ------------  -----------  ------------  -------
      1        8.8045       0.5120        1.0602  11.5880
      2        1.2264       0.5120        1.0871  11.5680
      3        1.0981       0.5120        1.0788  11.4270
      4        1.0841       0.5120        1.0676  11.3356
0.5104333868378812
[[  0   0 310]
 [  0   0 300]
 [  0   0 636]]


In [54]:
# #test images with batch_size = 4 in test_loader

# names = ('mask', 'no-mask', 'not-person')

# _, _, _, test_loader = data_loader()
# dataiter = iter(test_loader)
# images, labels = dataiter.next()

# print("labels: ", labels)
# print("labels: ",labels[1])
# # 打印图片
# imshow(torchvision.utils.make_grid(images))
# print('GroundTruth: ', ' '.join('%5s' % names[labels[j]] for j in range(4)))


# outputs = model(images)
# _, predicted = torch.max(outputs, 1)
# print('Predicted: ', ' '.join('%5s' % names[predicted[j]] for j in range(4)))

